In [1]:
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf

import datetime, os
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle
import random

import nengo_dl

# Deep Learning Nengo Network

Tensorflow version 2.2.0 <br>
Nengo dl version 3.0.0

In [2]:
class FeedForwardTrain(pytry.Trial):
    def params(self):
        self.param('run_n', run_n=1)
        self.param('theta', theta=3)
        self.param('q', q=4)
    
    def evaluate(self, param):       
        #LOAD/SET PARAMETERS
        run_n=param.run_n     
        theta=param.theta  
        q=param.q  
        learn_rate=0.000001 #learning rate
        n_epoch=1000 #number of training epochs
        train_rat=0.7 #ratio of training to testing data
        
        print(run_n) #print run number
        np.random.seed() #set seed to random value
        
#################################################################
        #Set and create data directories
        data_dir="D:\\NEN002\\GridSearch_NDL_LMU\\"+str(q)+'_'+str(theta)+"\\" #directory to save data to
        
        if run_n == 0:
            os.mkdir(data_dir)
            os.mkdir("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(q)+'_'+str(theta)+"\\")
            
        logdir = os.path.join("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(q)+'_'+str(theta))+"\\"+str(run_n) #directory to save logs to for tensorboard
        
        #load data
        highengdata = np.load('high_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('higheng.npy', allow_pickle=True)
        lowengdata = np.load('low_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('loweng.npy', allow_pickle=True)
##################################################################

        high=list(highengdata[:])
        low=list(lowengdata[:])

        #shuffle clips
        np.random.shuffle(high)
        np.random.shuffle(low)

        #get training clips and convert to list of frames
        high_train = high[:(int(len(low)*train_rat))]
        high_train=list(np.vstack(high_train))
        low_train = low[:(int(len(low)*train_rat))]
        low_train=list(np.vstack(low_train))

        #shuffle frames
        np.random.shuffle(high_train)
        np.random.shuffle(low_train)

        #Make each training set same size (low is shortest)
        high_train = high_train[:(int(len(low_train)))]
        low_train = low_train[:(int(len(low_train)))]

        #Create 10% validation dataset
        high_val = high[(int(len(low)*0.9)):(int(len(low)*1))]
        low_val = low[(int(len(low)*0.9)):(int(len(low)*1))]
        
        #convert to list of frames
        high_val=list(np.vstack(high_val))
        low_val=list(np.vstack(low_val))

        #shuffle frames
        np.random.shuffle(high_val)
        np.random.shuffle(low_val)
        
        #create test sets
        #get 30% of clips for testing
        high_test = high[(int(len(low)*train_rat)):(int(len(low)*0.9))]
        low_test = low[(int(len(low)*train_rat)):(int(len(low)*0.9))]

        #shuffle frames
        np.random.shuffle(high_test)
        np.random.shuffle(low_test)

        #GENERATE INPUT MATRICES
        #Concatenate high and low sets together to create a single array for training, test and validation separately
        all_train = np.vstack(np.concatenate((high_train, low_train)))
        all_test = np.concatenate((high_test, low_test)) 
        all_val = np.concatenate((high_val, low_val))

        #create the target data for training
        target_train = np.zeros((all_train.shape[0],2))
        n_high = len(high_train)
        target_train[:n_high,0] = 1 #target for high = [1,0]
        target_train[n_high:,1] = 1 #target for low = [0,1]
        
        #and for validation
        target_val = np.zeros((all_val.shape[0],2))
        n_high = len(high_val)
        target_val[:n_high,0] = 1
        target_val[n_high:,1] = 1
        
        #SAVE TRAINING AND TESTING DATA
        pickle_filename = (data_dir+"/%s_training_data.pkl") % str(run_n) #(data_dir+"/%s_training_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_train, file)

        pickle_filename = (data_dir+"/%s_testing_data.pkl") % str(run_n) #(data_dir+"/%s_testing_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_test, file)

        #BUILD MODEL (input layer -> hidden layer -> output layer)
        N = 200
        seed = 1
        N_dims = len(all_train[0]) 

        model = nengo.Network(seed=seed)
        with model:    
            input = nengo.Node(np.zeros(N_dims))
            hidden1 = nengo.Ensemble(n_neurons=N, dimensions=N_dims, radius=np.sqrt(N_dims), 
                                     neuron_type=nengo.RectifiedLinear())
            nengo.Connection(input, hidden1, synapse=None)
            output = nengo.Node(None, size_in=2)
            nengo.Connection(hidden1, output, eval_points=all_train, function=target_train, 
                             scale_eval_points=False, synapse=None)

            p_output = nengo.Probe(output)
        
        #TRAINING DATA AND PARAMETERS
        minibatch_size = 1000       # this can be adjusted to speed up training
                                    # Note: with larger minibatch_size, you may need to reduce this
                                    #(if training is making things worse, then you need to reduce this!)
                                    # number of iterations through the training data to perform

        batches = int(np.ceil(len(all_train)/minibatch_size)) #number of training batches needed (length of data / minibatch size)
        val_batches = int(np.ceil(len(all_val)/minibatch_size)) #number of validation batches needed

        training_data_input = np.array(all_train, copy=True)
        order = np.arange(len(training_data_input))
        np.random.shuffle(order)
        training_data_input_rand = training_data_input[order]
        training_data_input_rand.resize(minibatch_size,batches,N_dims)
        training_data_input.resize(minibatch_size,batches,N_dims)

        training_data_output = np.array(target_train, copy=True)
        training_data_output_rand = training_data_output[order]
        training_data_output.resize(minibatch_size,batches,2)
        training_data_output_rand.resize(minibatch_size,batches,2)
        
        val_input = np.array(all_val, copy=True)
        order = np.arange(len(val_input))
        np.random.shuffle(order)
        val_input_rand = val_input[order]
        val_input_rand.resize(minibatch_size,val_batches,N_dims)
        val_input.resize(minibatch_size,val_batches,N_dims)
        
        val_output = np.array(target_val, copy=True)
        val_output_rand = val_output[order]
        val_output.resize(minibatch_size,val_batches,2)
        val_output_rand.resize(minibatch_size,val_batches,2)
        
        #set learning objective (defines loss)
        def objective(outputs, targets): 
            return tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
                logits=outputs, labels=targets)  
        
        #TRAINING
        with nengo_dl.Simulator(
                model, minibatch_size=minibatch_size) as sim:
            sim.compile(optimizer=tf.optimizers.Adam(learn_rate),
                loss={p_output: objective})
            sim.fit(training_data_input_rand, training_data_output_rand, epochs=n_epoch,
                   callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir)],
                   validation_data=(val_input_rand, val_output_rand))
            sim.save_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n #('./trained.data') 
            
        #Run model with training data, using trained weights
        input.output = nengo.processes.PresentInput(all_train, presentation_time=0.001)
        with nengo_dl.Simulator(model, minibatch_size=1) as sim:
            sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
            sim.run(len(all_train)*0.001)
            
        #Save data
        pickle_filename = (data_dir+"/%s_train_target.pkl") % str(run_n) #(data_dir+"/%s_train_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_train, file)
        
        pickle_filename = (data_dir+"/%s_train_trange.pkl") % str(run_n) #(data_dir+"/%s_train_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_train_out.pkl") % str(run_n) #(data_dir+"/%s_train_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.data[p_output][0], file)    
        
        test_out=[]
        target_test=[]
        for j in range(len(all_test)):
            if j<int(len(high_test)):
                target = np.zeros((all_test[j].shape[0],2))
                target[:,0] = 1
            else:
                target = np.zeros((all_test[j].shape[0],2))
                target[:,1] = 1
            input.output = nengo.processes.PresentInput(all_test[j], presentation_time=0.001)
            with nengo_dl.Simulator(model, minibatch_size=1) as sim:
                sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
                sim.run(len(all_test[j])*0.001)
            test_out.append(sim.data[p_output][0])
            target_test.append(target)
            
        #Save data
        pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run_n) #(data_dir+"/%s_test_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_test, file)
        
        pickle_filename = (data_dir+"/%s_test_trange.pkl") % str(run_n) #(data_dir+"/%s_test_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run_n) #(data_dir+"/%s_test_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(test_out, file)


In [ ]:
#for run in range(5):
for theta in (1,3,5,7): #1,3,5,7
    FeedForwardTrain().run(run_n=5, q=2, theta=theta)

In [ ]:
FeedForwardTrain().run(run_n=5, q=2, theta=7)

running FeedForwardTrain#20200420-132311-f25b6872
5
|#####################Building network (57%)                     | ETA: 0:00:00

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. It is recommended that you install tensorflow-gpu (`pip install tensorflow-gpu`).
  "No GPU support detected. It is recommended that you "


Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Train on 1000 samples, validate on 1000 samples


Epoch 1/1000
1000/1000 [==============================] - 5s 5ms/sample - loss: 0.7936 - probe_loss: 0.7936 - val_loss: 0.8038 - val_probe_loss: 0.8038
Epoch 2/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7774 - probe_loss: 0.7774 - val_loss: 0.7876 - val_probe_loss: 0.7876
Epoch 3/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7612 - probe_loss: 0.7612 - val_loss: 0.7714 - val_probe_loss: 0.7714
Epoch 4/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7449 - probe_loss: 0.7449 - val_loss: 0.7552 - val_probe_loss: 0.7552
Epoch 5/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7287 - probe_loss: 0.7287 - val_loss: 0.7389 - val_probe_loss: 0.7389
Epoch 6/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7125 - probe_loss: 0.7125 - val_loss: 0.7227 - val_probe_loss: 0.7227
Epoch 7/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.6963 -

Epoch 55/1000
1000/1000 [==============================] - 4s 4ms/sample - loss: -0.0819 - probe_loss: -0.0819 - val_loss: -0.0723 - val_probe_loss: -0.0723
Epoch 56/1000
1000/1000 [==============================] - 4s 4ms/sample - loss: -0.0981 - probe_loss: -0.0981 - val_loss: -0.0885 - val_probe_loss: -0.0885
Epoch 57/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1144 - probe_loss: -0.1144 - val_loss: -0.1048 - val_probe_loss: -0.1048
Epoch 58/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1306 - probe_loss: -0.1306 - val_loss: -0.1210 - val_probe_loss: -0.1210
Epoch 59/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1468 - probe_loss: -0.1468 - val_loss: -0.1372 - val_probe_loss: -0.1372
Epoch 60/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1630 - probe_loss: -0.1630 - val_loss: -0.1534 - val_probe_loss: -0.1534
Epoch 61/1000
1000/1000 [==============================] -

...

Epoch 628/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -9.3726 - probe_loss: -9.3726 - val_loss: -9.3685 - val_probe_loss: -9.3685
Epoch 629/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -9.3888 - probe_loss: -9.3888 - val_loss: -9.3847 - val_probe_loss: -9.3847
Epoch 630/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -9.4051 - probe_loss: -9.4051 - val_loss: -9.4009 - val_probe_loss: -9.4009
Epoch 631/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -9.4213 - probe_loss: -9.4213 - val_loss: -9.4172 - val_probe_loss: -9.4172
Epoch 632/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -9.4375 - probe_loss: -9.4375 - val_loss: -9.4334 - val_probe_loss: -9.4334
Epoch 633/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -9.4537 - probe_loss: -9.4537 - val_loss: -9.4496 - val_probe_loss: -9.4496
Epoch 634/1000
1000/1000 [==========================

Epoch 680/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -10.2158 - probe_loss: -10.2158 - val_loss: -10.2120 - val_probe_loss: -10.2120
Epoch 681/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -10.2320 - probe_loss: -10.2320 - val_loss: -10.2283 - val_probe_loss: -10.2283
Epoch 682/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -10.2483 - probe_loss: -10.2483 - val_loss: -10.2445 - val_probe_loss: -10.2445
Epoch 683/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -10.2645 - probe_loss: -10.2645 - val_loss: -10.2607 - val_probe_loss: -10.2607
Epoch 684/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -10.2807 - probe_loss: -10.2807 - val_loss: -10.2769 - val_probe_loss: -10.2769
Epoch 685/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -10.2969 - probe_loss: -10.2969 - val_loss: -10.2931 - val_probe_loss: -10.2931
Epoch 686/1000
1000/1000 [==

Epoch 731/1000
1000/1000 [==============================] - 4s 4ms/sample - loss: -11.0428 - probe_loss: -11.0428 - val_loss: -11.0393 - val_probe_loss: -11.0393
Epoch 732/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.0590 - probe_loss: -11.0590 - val_loss: -11.0555 - val_probe_loss: -11.0555
Epoch 733/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -11.0752 - probe_loss: -11.0752 - val_loss: -11.0718 - val_probe_loss: -11.0718
Epoch 734/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.0915 - probe_loss: -11.0915 - val_loss: -11.0880 - val_probe_loss: -11.0880
Epoch 735/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -11.1077 - probe_loss: -11.1077 - val_loss: -11.1042 - val_probe_loss: -11.1042
Epoch 736/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -11.1239 - probe_loss: -11.1239 - val_loss: -11.1204 - val_probe_loss: -11.1204
Epoch 737/1000
1000/1000 [==

Epoch 782/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.8698 - probe_loss: -11.8698 - val_loss: -11.8666 - val_probe_loss: -11.8666
Epoch 783/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.8860 - probe_loss: -11.8860 - val_loss: -11.8828 - val_probe_loss: -11.8828
Epoch 784/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -11.9022 - probe_loss: -11.9022 - val_loss: -11.8990 - val_probe_loss: -11.8990
Epoch 785/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.9185 - probe_loss: -11.9185 - val_loss: -11.9153 - val_probe_loss: -11.9153
Epoch 786/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -11.9347 - probe_loss: -11.9347 - val_loss: -11.9315 - val_probe_loss: -11.9315
Epoch 787/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -11.9509 - probe_loss: -11.9509 - val_loss: -11.9477 - val_probe_loss: -11.9477
Epoch 788/1000
1000/1000 [==

Epoch 833/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -12.6968 - probe_loss: -12.6968 - val_loss: -12.6938 - val_probe_loss: -12.6938
Epoch 834/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -12.7130 - probe_loss: -12.7130 - val_loss: -12.7101 - val_probe_loss: -12.7101
Epoch 835/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -12.7293 - probe_loss: -12.7293 - val_loss: -12.7263 - val_probe_loss: -12.7263
Epoch 836/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -12.7455 - probe_loss: -12.7455 - val_loss: -12.7425 - val_probe_loss: -12.7425
Epoch 837/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -12.7617 - probe_loss: -12.7617 - val_loss: -12.7587 - val_probe_loss: -12.7587
Epoch 838/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -12.7779 - probe_loss: -12.7779 - val_loss: -12.7749 - val_probe_loss: -12.7749
Epoch 839/1000
1000/1000 [==

To view logs in tensorboard use following command in command prompt: <br>
    tensorboard --logdir={logs_base_dir} --host=localhost <br>
<br>
Note: change {logs_base_dir} to the name of the folder where logs are being saved. <br>
Note: make sure you have navigated to the directory containing this script in the command prompt before running the above command.